In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft

In [2]:
AFM_FOLDER = "20200818_Akiyama_AFM/"

In [3]:
df = sio.read_dat(AFM_FOLDER + "frq-sweep001.dat")
freq_shift = df["Frequency Shift (Hz)"]
amplitude = df["Amplitude (m)"] / 500
phase = df["Phase (deg)"]
fano = sft.fit_fano(freq_shift, amplitude)
lorentzian = sft.fit_lorentzian(freq_shift, phase, linear_offset=False)

In [4]:
f_res = 44379.7064
sigma = 62.2841355
print(f_res/sigma)

A_drive = 50e-3
A_res = 28.3e-6 * 1 / 500e-6
print(A_res/A_drive)

# Calibration
A_drive = 50e-3
osc_amp = 50e-9

print(osc_amp/A_drive)

712.5362830154398
1.132
1e-06


In [5]:
%matplotlib widget

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.plot(freq_shift, amplitude)
ax1.plot(freq_shift, fano.best_fit)
ax1.set_ylabel(df.columns[2])

ax2.plot(freq_shift, phase)
ax2.plot(freq_shift, lorentzian.best_fit)
ax2.set_ylabel(df.columns[3])
ax2.set_xlabel(df.columns[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Center Frequency (Hz)')

In [6]:
fano

In [7]:
lorentzian